### Calculation of PER using machine learning

**What is PER?**  
The Price-to-Earnings ratio (P/E ratio) in finance is a key measure used to assess the valuation of a company in the stock market.  
The PER provides an indication of the number of years it would take to recoup the initial investment if the company were to distribute all its profits to shareholders as dividends.  
A high PER may indicate that the market values the company at a higher level relative to its current earnings, which may imply that investors anticipate future growth. Conversely, a low PER may indicate that the company is undervalued relative to its current earnings, which may represent an investment opportunity.

**How is PER calculated?**  
$$PER = Stock Price / Earnings Per Share$$

**Classic PER analysis:**  
To determine if a company has a high or low PER, it should be compared to the average PER of companies in the same sector.  
If our company has a PER lower (or higher) than the average, then our company is undervalued (or overvalued).

**PER analysis with machine learning:**  
In our study, to find comparable companies, we will consider:
- *The company's sector*
- *Market capitalization*
- *Total assets*
- *Revenue*

We will use a machine learning model to determine what should be the PER of our company according to the aforementioned criteria.  
As input, we will use the logarithm of market capitalization, total assets, and revenue (log-transformed for a normal distribution), as well as the company's sector (dummy variable takes 1 for the studied company's sector and 0 otherwise).  
And as Output, the logarithm of the PER (which we will exponentiate to obtain our predicted PER).

Once our model is trained on a sufficiently large sample, we can calculate what should be the PER of the company according to our machine learning model and compare it with the actual PER of the company.  
- If Actual PER < Predicted PER: action undervalued  
- If Actual PER > Predicted PER: action overvalued

Then we will look at the median of spreads by sector to determine which sector would be rather undervalued or overvalued:  
$spread = predicted PER - actual PER$  
- If spread < 0: sector undervalued  
- If spread > 0: sector overvalued

**Results:** the mean absolute error (MAE) of our machine learning model is relatively high (around 8.5) but is more accurate than the sector average comparison method (around 9).

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, regularizers
import pandas as pd
import numpy as np
from yahooquery import Ticker
from tqdm import tqdm

In [ ]:
# We load our dataset with information about companies
df_dataset = pd.read_csv("Dataset French Companies.csv")

list_tickers = df_dataset["Ticker"].tolist()
stocks_data = Ticker(list_tickers[:10])

In [ ]:
# I created a progress bar because it takes some time to download the required data
with tqdm(total=100, desc="Starting...") as progress_bar:

    progress_bar.set_description("Phase 1: Requesting MarketCap and PER")
    # We retrieve the market_cap and the current PER
    df_valuation_measures = stocks_data.valuation_measures[["asOfDate", "periodType", "MarketCap", "PeRatio"]]
    df_valuation_measures.sort_values(by="asOfDate", inplace=True)

    # We keep only the value TTM to make sure that all the data for each stocks are time-standardized
    df_valuation_measures = df_valuation_measures.query("periodType == 'TTM'")

    # We retrieve the marketcap
    # And we keep the last value of each group => that is the most recent value since we had sorted the df previously
    df_market_cap = df_valuation_measures.dropna(subset=["MarketCap"]).groupby(level=0).last()["MarketCap"]

    # Same process to retrieve the most recent PER
    df_per = df_valuation_measures.dropna(subset=["PeRatio"]).groupby(level=0).last()["PeRatio"]
    progress_bar.update(20)


    progress_bar.set_description("Phase 2: Requesting TotalAsset")
    # We retrieve the Total assets
    df_total_assets = stocks_data.balance_sheet()[["asOfDate", "periodType", "TotalAssets"]]
    df_total_assets.sort_values(by="asOfDate", inplace=True)

    # We keep only the value TTM to make sure that all the data for each stocks are time-standardized
    df_total_assets = df_total_assets.query("periodType == '12M'")

    # We retrieve the value of TotalAssets
    # And we keep the last value of each group => that is the most recent value since we had sorted the df previously
    df_total_assets = df_total_assets.dropna(subset=["TotalAssets"]).groupby(level=0).last()["TotalAssets"]
    progress_bar.update(40)


    progress_bar.set_description("Phase 3: Requesting TotalRevenu")
    # We retrieve the total Revenue
    df_total_revenue = stocks_data.income_statement()[["asOfDate", "periodType", "TotalRevenue"]]
    df_total_revenue.sort_values(by="asOfDate", inplace=True)

    # We keep only the value TTM to make sure that all the data for each stocks are time-standardized
    df_total_revenue = df_total_revenue.query("periodType == '12M'")

    # We retrieve the value of TotalRevenue
    # And we keep the last value of each group => that is the most recent value since we had sorted the df previously
    df_total_revenue = df_total_revenue.dropna(subset=["TotalRevenue"]).groupby(level=0).last()["TotalRevenue"]
    progress_bar.update(40)

Phase 3: Requesting TotalRevenu: 100%|██████████| 100/100 [00:09<00:00, 10.25it/s]     


In [137]:
df_data = pd.concat([df_market_cap, df_per, df_total_assets, df_total_revenue], axis=1).astype(float)
df_data

,MarketCap,PeRatio,TotalAssets,TotalRevenue
symbol,,,,
AAA.PA,1.778655e+08,30.307692,3.805900e+08,3.656020e+08
AB.PA,5.264038e+07,NaN,2.549900e+07,9.700000e+05
ABCA.PA,2.909644e+08,17.594255,1.721100e+08,3.974800e+07
ABEO.PA,7.138495e+07,28.279954,3.035340e+08,2.483900e+08
ABNX.PA,4.678427e+07,9.944444,1.483400e+07,4.640000e+06
...,...,...,...,...
WAVE.PA,1.159213e+09,17.398524,9.777410e+08,7.010560e+08
WLN.PA,1.865549e+09,54.765109,2.173190e+10,4.610400e+09
XFAB.PA,5.564920e+08,5.356394,1.703814e+09,9.067860e+08


In [ ]:
df_data = pd.concat([df_market_cap, df_per, df_total_assets, df_total_revenue], axis=1).astype(float)
# We filter to remove negative values from our df because we are going to apply log on the data
df_data = df_data[df_data > 0]

# We remove extreme values (60 is arbitrary)
df_data = df_data.query("PeRatio < 60")

# We use the log to improve the efficiency of our ML model
log_data = np.log(df_data)

# We retrieve the name and the sector of the company
log_data["Name"] = log_data.index.map(df_dataset.set_index("Ticker")["Name"].to_dict())
log_data["Sector"] = log_data.index.map(df_dataset.set_index("Ticker")["Sector"].to_dict())

# We assign a dummy variable to the sector for it to be taken into account in our model
df_final = pd.concat([log_data, pd.get_dummies(log_data["Sector"])], axis=1)

# We remove the companies for which we are missing a data
df_final.dropna(inplace=True)
df_final

,MarketCap,PeRatio,TotalAssets,TotalRevenue,Name,Sector,Biens de consommation,Industries,Matériaux de base,Pétrole et Gaz,Santé,Services aux Collectivités,Services aux consommateurs,Sociétés financières,Technologies,Télécommunications
symbol,,,,,,,,,,,,,,,,
AAA.PA,18.996538,3.411402,19.757233,19.717056,ALAN ALLMAN ASSOCIATES,Sociétés financières,0,0,0,0,0,0,0,1,0,0
ABCA.PA,19.488711,2.867572,18.963644,17.498070,ABC ARBITRAGE,Sociétés financières,0,0,0,0,0,0,0,1,0,0
ABEO.PA,18.083598,3.342153,19.531004,19.330511,ABEO,Biens de consommation,1,0,0,0,0,0,0,0,0,0
ABNX.PA,17.661058,2.297014,16.512432,15.350225,ABIONYX PHARMA,Santé,0,0,0,0,1,0,0,0,0,0
AC.PA,23.078776,2.952095,23.145144,22.343841,ACCOR,Services aux consommateurs,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VU.PA,21.531505,3.554707,20.773907,20.502567,VUSIONGROUP,Industries,0,1,0,0,0,0,0,0,0,0
WAVE.PA,20.871007,2.856385,20.700755,20.368098,WAVESTONE,Technologies,0,0,0,0,0,0,0,0,1,0
WLN.PA,21.346821,4.003053,23.802047,22.251580,WORLDLINE,Industries,0,1,0,0,0,0,0,0,0,0


In [ ]:
# Our explanatory variable of our model are the sector, the market cap, the total asset and the revenues
explanatory_var = list(set(df_final["Sector"])) + ["MarketCap", "TotalAssets", "TotalRevenue"]
X = df_final[explanatory_var]
# And the value we want to predict is the PER
y = df_final["PeRatio"]

# We split our data to use 80% of them to train our model, and 20% of them to test it
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

stock_model = tf.keras.Sequential([
    layers.Dense(len(explanatory_var), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(len(explanatory_var)//2, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(1) # Because we need only one value as an output : the predicted PER
])

stock_model.compile(loss=tf.keras.losses.mae,
                   optimizer=tf.keras.optimizers.Adamax(),
                   metrics=["mae"])

stock_model.fit(X_train, y_train, epochs=50, verbose=0)

# We test our model on our test data (that were not use for the training)
stock_model.evaluate(X_test, y_test)[1]

3/3 [==============================] - 0s 7ms/step - loss: 0.8278 - mae: 0.7444


0.7444040775299072

In [114]:
# We apply our machine learning model to each company in the dataset to calculate their predicted PER
predicted_per_raw = stock_model.predict(df_final[explanatory_var])
# We format our result for it to be readable
df_predicted_per = pd.DataFrame(predicted_per_raw).rename(columns={0:"PredictedPER"})
df_predicted_per.index = df_final.index
# We concat in a same df the actual PER and the predicted PER to compare them
df_compare = pd.concat([df_final[["Name", "Sector", "PeRatio"]], df_predicted_per], axis=1)

# We replace the ticker with the Name in index for the df to be more readable
df_compare.set_index("Name", inplace=True)

# We calculate the exp of the log PERs
df_compare[["PeRatio", "PredictedPER"]] = np.exp(df_compare[["PeRatio", "PredictedPER"]])

# We calculate the mean of the PER per sector
mean_PER_sector = df_compare.groupby("Sector")["PeRatio"].mean()

# And we add the sector mean in the df for each company
df_compare["Sector_mean"] = df_compare["Sector"].map(mean_PER_sector)

# Calculate the spread between the actual PER and the predicted PER with our model
df_compare["Spread_ML_model"] = df_compare["PeRatio"] - df_compare["PredictedPER"]

# Calculate the spread between the actual PER and sector_mean
df_compare["Spread_sector_mean"] = df_compare["PeRatio"] - df_compare["Sector_mean"]
df_compare

13/13 [==============================] - 0s 3ms/step


,Sector,PeRatio,PredictedPER,Sector_mean,Spread_ML_model,Spread_sector_mean
Name,,,,,,
ALAN ALLMAN ASSOCIATES,Sociétés financières,30.307692,8.335178,13.151469,21.972514,17.156223
ABC ARBITRAGE,Sociétés financières,17.594255,11.724759,13.151469,5.869496,4.442786
ABEO,Biens de consommation,28.279954,8.265483,13.580502,20.014471,14.699452
ABIONYX PHARMA,Santé,9.944444,11.430606,17.687624,-1.486162,-7.743180
ACCOR,Services aux consommateurs,19.146018,16.424103,13.950604,2.721915,5.195414
...,...,...,...,...,...,...
VUSIONGROUP,Industries,34.977578,19.235483,19.898506,15.742095,15.079072
WAVESTONE,Technologies,17.398524,17.704147,18.423709,-0.305623,-1.025185
WORLDLINE,Industries,54.765109,18.820427,19.898506,35.944682,34.866603


## Conclusion and analysis

In [116]:
# We create a top of the most undervalued and overvalued companies based on our model
df_top_companies = df_compare[["PeRatio", "PredictedPER", "Spread_ML_model"]].copy()
df_top_companies["Status"] = np.where(df_top_companies["Spread_ML_model"] > 0, "OverValued", "UnderValued")
df_top_companies.sort_values("Spread_ML_model")

,PeRatio,PredictedPER,Spread_ML_model,Status
Name,,,,
CREDIT AGRICOLE SA,6.274650,25.317076,-19.042426,UnderValued
BNP PARIBAS,6.636710,24.566154,-17.929444,UnderValued
TRANSITION EVERGREEN,6.127660,23.399389,-17.271729,UnderValued
EURAZEO,2.996870,20.258732,-17.261862,UnderValued
COVIVIO,4.420455,19.552004,-15.131549,UnderValued
...,...,...,...,...
I2S,54.479419,9.603827,44.875592,OverValued
BIO-UV GRP,55.026110,9.750913,45.275197,OverValued
ICADE,59.605634,13.461471,46.144163,OverValued


In [125]:
# We calculate whether the sectors are under- or overvalued based on our model => calculate the median of the spread
df_spread_sector = df_compare.groupby("Sector")["Spread_ML_model"].median().rename("Spread_median").to_frame().sort_values("Spread_median")
df_spread_sector["Status"] = np.where(df_spread_sector["Spread_median"] > 0, "OverValued", "UnderValued")
df_spread_sector

,Spread_median,Status
Sector,,
Pétrole et Gaz,-4.989449,UnderValued
Sociétés financières,-4.172450,UnderValued
Télécommunications,-2.147011,UnderValued
Biens de consommation,-1.320450,UnderValued
Services aux consommateurs,1.509044,OverValued
Services aux Collectivités,1.550466,OverValued
Technologies,1.670685,OverValued
Industries,1.709110,OverValued
Santé,2.707996,OverValued


In [ ]:
# To compare both technics we calculate the absolute mean, instead of the simple mean because opposite values could cancel themselves, though they could be large, they woudln't be taken into account
spread_mean_ML_model = df_compare["Spread_ML_model"].abs().mean().round(2)
spread_mean_sector_avg = df_compare["Spread_sector_mean"].abs().mean().round(2)

if spread_mean_ML_model < spread_mean_sector_avg :
    print("Our ML model is more efficient than the traditional comparison to the sector average.")
else : 
    print("Our model needs to be improved to be more accurate than the traditional comparison to the sector average.")
print("We want the spread to be the smaller.")
print(f"Spread median of our model: {spread_mean_ML_model} \nSpread mean of the sector average method: {spread_mean_sector_avg}")

Our ML model is more efficient than the traditional comparison to the sector average.
We want the spread to be the smaller.
Spread median of our model: 8.69 
Spread mean of the sector average method: 9.16
